# Lab 2: Memory Integration (15 minutes)
In this Lab, you will look at core concepts such as conversation history, agent state and request state which help you understand short term memory of agents, to start building tailored agents. You will also learn how to integrate mem0.io for long term memory and store user preferences and maintain context in longer multi-turn conversations. 

## Learning Objectives
- ✅ Understand short-term built-in Strands Agents memory
- ✅ Integrate long-term memory using mem0.io
- ✅ Store user preferences and reuse them to customize agent responses

> ⚠️ **EDUCATIONAL PURPOSE ONLY**: This lab demonstrates budget analysis and there are conversations with the agent about how to save and budget money. This is NOT financial advice and the soundness of the agent outputs should be verified against expert advice. All budget analysis is for educational demonstration of AI agents only.

## Step 1: Setup

In [ ]:
# Install Strands using pip

!pip install -q strands-agents strands-agents-tools mcp

from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import json
from datetime import datetime

print("📚 Loading Strands SDK...")

# Configure AWS Bedrock model (same as Lab 1)
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

## Step 2: Understanding State Management and Agent Memory

Understanding how state works in Strands is essential for building agents that can maintain context across multi-turn interactions and workflows

### Short Term Memory
**Strands Agents state** is maintained in several forms:

1. Conversation History: The sequence of messages between the user and the agent.

Strands uses a conversation manager to handle conversation history effectively. The default is the `SlidingWindowConversationManager`, which keeps recent messages and removes older ones when needed
   
2. Agent State: Stateful information outside of conversation context such as temporary variables and state, maintained across multiple requests.

   
3. Request State: Contextual information maintained within a single request.

Conversation history, agent state and request state form the short term memory of the Agent, which helps the LLM with immediate context to maintain the conversation.



#### 2.1 Short Term memory: Conversation History

In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

# Create a conversation manager with custom window size
# By default, SlidingWindowConversationManager is used even if not specified
conversation_manager = SlidingWindowConversationManager(
    window_size=10,  # Maximum number of message pairs to keep
)

# Use the conversation manager with your agent
memory_demo_agent = Agent(
    model=model,
    system_prompt="You are a financial assistant. Keep responses concise.",
    conversation_manager=conversation_manager
)

### 📝 Test the conversational history of the Agent. Does the Agent retain context it learnt from earlier messages after message 10?

⚠️ You can engage in multi-turn conversation and to break out of the chat, type "exit"

In [ ]:
# Example prompt: "I want to save $800 per month and focus on reducing my dining expenses"

# Interactive loop
while True:
    try:
        user_input = input("\n> ")

        if user_input.lower() == "exit":
            print("\nGoodbye! 👋")
            break

        # Call the memory agent
        memory_demo_agent(user_input)
        print(f"\n📝 Memory now: {len(memory_demo_agent.messages)} messages")

    except KeyboardInterrupt:
        print("\n\nExecution interrupted. Exiting...")
        break
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("Please try a different request.")

#### The `agent.messages` list contains all user and assistant messages, including tool calls and tool results. This is the primary way to inspect what's happening in your agent's conversation and check it's short term memory.

In [ ]:
memory_demo_agent.messages

#### 2.2 Short Term memory: Agent State

Agent state provides key-value storage for stateful information that exists outside of the conversation context. Unlike conversation history, agent state is not passed to the model during inference but can be accessed and modified by tools and application logic

In [ ]:
# Create an agent with initial state
memory_demo_agent = Agent(
    model=model,
    system_prompt="You are a financial assistant. Keep responses concise.",
    state={"user_preferences": {"saving_goal": "40%", "spending_goal":"20%", "fixed_expenses":"40%"}},
    conversation_manager=conversation_manager
)

# Access state values
user_finance_goals = memory_demo_agent.state.get("user_preferences")
print(user_finance_goals) 

# Set new state values
memory_demo_agent.state.set("session_count", 0)

# Get entire state
all_state = memory_demo_agent.state.get()
print(all_state)  # All state data as a dictionary

# Delete state values
memory_demo_agent.state.delete("last_action")

🎯 **By storing information in the state, you can reuse it while designing tools for agents to customise the agent behavior for users**

#### 2.3 Short Term memory: Request State

Each agent interaction maintains a request state dictionary that persists throughout the event loop cycles and is not included in the agent's context.

The request state:

- Is initialized at the beginning of each agent call
- Persists through recursive event loop cycles
- Can be modified by callback handlers
- Is returned in the AgentResult object

In [ ]:
def custom_callback_handler(**kwargs):
    # Access request state
    if "request_state" in kwargs:
        state = kwargs["request_state"]
        # Use or modify state as needed
        if "counter" not in state:
            state["counter"] = 0
        state["counter"] += 1
        print(f"Callback handler event count: {state['counter']}")

memory_demo_agent = Agent(
    model=model,
    system_prompt="You are a financial assistant. Keep responses concise.",
    state={"user_preferences": {"saving_goal": "40%", "spending_goal":"20%", "fixed_expenses":"40%"}},
    conversation_manager=conversation_manager,
    callback_handler=custom_callback_handler
)

result = memory_demo_agent("Hi there!")

print(result.state)

## Long Term Memory: Personalized Context Through Persistent Memory

Now, let's integrate mem0.io and create a memory agent that can store user preferences and hold memories.

**Memory Backend Options**
The Mem0 Memory Tool supports three different backend configurations:

1. OpenSearch (Recommended for AWS environments):

- Requires AWS credentials and OpenSearch configuration
- Set OPENSEARCH_HOST and optionally AWS_REGION (defaults to us-west-2)

2. FAISS (Default for local development):

- Uses FAISS as the local vector store backend
- Requires faiss-cpu package for local vector storage
- No additional configuration needed

3. Mem0 Platform:

- Uses the Mem0 Platform API for memory management
- Requires a Mem0 API key : MEM0_API_KEY in the environment variables

In [ ]:
# For CPU version
!pip install -q faiss-cpu mem0ai opensearch-py

In [ ]:
import os
import logging
from dotenv import load_dotenv

from strands import Agent
from strands_tools import mem0_memory, use_llm

logger = logging.getLogger(__name__)

# Load environment variables from .env file if it exists
load_dotenv()

# We are using the default FAISS vector store for this demo
USER_ID = "mem0_user"

### Tool Overview of Memory Agent
The memory agent utilizes two primary tools:

1. memory: Enables storing and retrieving information with capabilities for:

- Storing user-specific information persistently
- Retrieving memories based on semantic relevance
- Listing all stored memories for a user
- Setting relevance thresholds and result limits

2. use_llm: Provides language model capabilities for:

- Generating conversational responses based on retrieved memories
- Creating natural, contextual answers using memory context

In [ ]:
# System prompt for the memory agent
MEMORY_SYSTEM_PROMPT = f"""You are a personal finance assistant that maintains context by remembering user details.

Capabilities:
- Store new information using mem0_memory tool (action="store")
- Retrieve relevant memories (action="retrieve")
- List all memories (action="list")
- Provide personalized responses

Key Rules:
- Always include user_id={USER_ID} in tool calls
- Be conversational and natural in responses
- Format output clearly
- Acknowledge stored information
- Only share relevant information
- Politely indicate when information is unavailable
"""

In [ ]:
# Create an agent with memory capabilities
memory_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=MEMORY_SYSTEM_PROMPT,
    tools=[mem0_memory, use_llm]
)

In [ ]:
# Initialize some user preferences
def initialize_user_preferences():
    """Initialize user preferences."""
    
    content = """My name is Charlie. I prefer to have a monthly budget of 40% for fixed expenses, 30% for wants and 30% saved. 
    I am planning a trip to South Korea next spring and would like to dedicate some portion from the savings to a vacation budget which over 
    12 months should amount to $4000. 
    My favourite hobbies are visiting new restaurants and look for discounts, 
    new openings which help me visit more restaurants within my budget."""  # noqa
    memory_agent.tool.mem0_memory(action="store", content=content, user_id=USER_ID)


## Test the memory agent
- Check if the Agent provides responses with Charlie's preferences
- If you input 'demo', the user preferences for Charlie will be initialised in the memory
- To end the multi-turn conversation with the Agent, type 'exit'

### ⚠️ You can engage in multi-turn conversation and to break out of the chat, type "exit"

In [ ]:
# Interactive loop

# Sample_prompts: "I got a promotion and my monthly salary has changed to $4700 - 
# How does it affect my vacation savings for the South Korea trip?"

# Sample_prompt: "My wife earns $5000 and we have the same goals as a family. How early can we save for South Korea trip?"
while True:
    try:
        user_input = input("\n> ")

        if user_input.lower() == "exit":
            print("\nGoodbye! 👋")
            break
        elif user_input.lower() == "demo":
            initialize_user_preferences()
            print("\nUser preferences for Charlie initialized!")
            continue

        # Call the memory agent
        memory_agent(user_input)

    except KeyboardInterrupt:
        print("\n\nExecution interrupted. Exiting...")
        break
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("Please try a different request.")

## Summary

### What You've Learned:
- ✅ **Agent Short-Term Memory**: Conversartion History, agent state and request state
- ✅ **mem0.io Integration**: Strands memory enabled agent class for storing user preferences
- ✅ **Memory Based capabilities**: `mem0_memory` tool stores, retrieves and lists memories


### Key Takeaways:
1. Strands agents remembers messages through the conversation history
2. Agent state provides key-value storage for stateful information to be stored
3. Request state dictionary persists throughout the event loop cycles - Separate from agent context
4. Tools can access stored user preferences and context from historic messages
5. Memory makes agents truly personalized


### Next: Lab 3 - Multi-Agent Teams
Create multiple agents that work together!